Sometimes we just need to calculate factor for a specific day (e.g., daily incremental calculation for factor inventory), so the script below shows how to retrieve a day's factor value with minimal calcualtion.

The tricky part is that most factors are rolling-based, so in order to acquire non-nan factor value, one must use a lookback window to collect enough historical data for calculation. Determining the optimal length of this lookback window, however, poses a significant difficulty.

In [1]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

In [2]:
from frozen.data import DataLoadManager, DatabaseTypes
from frozen.factor.expression.utils import get_safe_lookback_period
from frozen.factor.expression.utils.tools import calc_str
from frozen.factor.expression.operators import *

In [3]:
string = 'rank(Ts_Max(SignedPower(where(returns < 0 ? stddev(returns, 5) : stddev(close, 5)), 2.0), 5))'

The `get_safe_lookback_period` method parses the string and gives the optimal lookback window without having to calculate a whole batch.

In [4]:
lookback = get_safe_lookback_period(string)

In [5]:
lookback

9

In [6]:
dataloader = DataLoadManager(database_type_or_config=DatabaseTypes.DUCKDB)

In [7]:
# close, pct_chg = dataloader.load_volume_price('stock_daily_hfq', ('close', 'pct_chg'), ("000001.SZ", "000002.SZ"), end_date="20230605", lookback=lookback)

# close = Factor(close, "close")
# returns = Factor(pct_chg, "returns")

In [8]:
data_definitions = [
    ('stock_daily_hfq', ('close', 'pct_chg'), ('close', 'returns')),
]

data = dataloader.load_batch(data_definitions, ("000001.SZ", "000002.SZ"), end_date="20230605", lookback=lookback)

In [9]:
calc_str(string, data, piecewise=True).data

ticker,000001.SZ,000002.SZ
2023-06-05,2.0,1.0


In [10]:
calc_str(string, data).data

ticker,000001.SZ,000002.SZ
trade_date,,
2023-05-23,NaN,NaN
2023-05-24,NaN,NaN
2023-05-25,NaN,NaN
2023-05-26,NaN,NaN
2023-05-29,NaN,NaN
2023-05-30,NaN,NaN
2023-05-31,NaN,NaN
2023-06-01,NaN,NaN
2023-06-02,2.0,1.0
